In [1]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
#from keras import backend as k 
#from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
from PIL import Image
from keras.models import load_model
from keras import models
from keras import layers
import pandas as pd
import numpy as np
import os
path = os.getcwd()
img_width, img_height = 256, 256
train_data_dir = os.path.join(path,'val256','train')
validation_data_dir = os.path.join(path,'val256','val')
test_dir=os.path.join(path,'test256')
nb_train_samples = 83
nb_validation_samples = 17 
batch_size = 1
epochs = 8

Using TensorFlow backend.


In [ ]:
#vgg = applications.VGG19(weights = "imagenet", include_top=False, input_shape = (img_width, img_height, 3))

In [ ]:
#downloading VGG model
vgg = applications.VGG16(weights = "imagenet", include_top=False, input_shape = (img_width, img_height, 3))

In [ ]:
#freezing last layers
for layer in vgg.layers[:-4]:
    layer.trainable = False

In [ ]:
#generation of data
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    horizontal_flip = True,
    fill_mode = "nearest",
    zoom_range = 0.3,
    width_shift_range = 0.3,
    height_shift_range=0.3,
    rotation_range=30)

val_datagen = ImageDataGenerator(
    rescale = 1./255,
    horizontal_flip = True,
    fill_mode = "nearest",
    zoom_range = 0.3,
    width_shift_range = 0.3,
    height_shift_range=0.3,
    rotation_range=30)


train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size = (img_height, img_width),
    batch_size = batch_size, 
    class_mode = "categorical")

validation_generator = val_datagen.flow_from_directory(
    validation_data_dir,
    target_size = (img_height, img_width),
    batch_size = batch_size, 
    class_mode = "categorical")


In [ ]:
#getting features from images (for train data and validation data)
train_features = np.zeros(shape=(nb_train_samples, 8, 8, 512))
train_labels = np.zeros(shape=(nb_train_samples,2))

validation_features = np.zeros(shape=(nb_validation_samples, 8, 8, 512))
validation_labels = np.zeros(shape=(nb_validation_samples,2))

i = 0
for inputs_batch, labels_batch in train_generator:
    features_batch = vgg.predict(inputs_batch)
    train_features[i * batch_size : (i + 1) * batch_size] = features_batch
    train_labels[i * batch_size : (i + 1) * batch_size] = labels_batch
    i += 1
    if i * batch_size >= nb_train_samples:
        break
i = 0
for inputs_batch, labels_batch in validation_generator:
    features_batch_val = vgg.predict(inputs_batch)
    validation_features[i * batch_size : (i + 1) * batch_size] = features_batch_val
    validation_labels[i * batch_size : (i + 1) * batch_size] = labels_batch
    i += 1
    if i * batch_size >= nb_validation_samples:
        break
         
train_features = np.reshape(train_features, (nb_train_samples, 8 * 8 * 512))
validation_features = np.reshape(validation_features, (nb_validation_samples, 8 * 8 * 512))

In [ ]:
#constructing model over VGG
model = models.Sequential()
model.add(vgg)
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(2, activation='softmax'))
model.summary()

In [ ]:
#fitting
model.compile(optimizer=optimizers.RMSprop(lr=1e-4),
              loss='categorical_crossentropy',
              metrics=['acc'])
 
history = model.fit_generator(
      train_generator,
      steps_per_epoch=train_generator.samples/train_generator.batch_size ,
      epochs=16,
      validation_data=validation_generator,
      validation_steps=validation_generator.samples/validation_generator.batch_size,
      verbose=1)

model.save('model.h5')

In [ ]:
#del model

In [2]:
model = load_model('model.h5')

In [4]:
sub=pd.read_csv('sample_submission.csv')

In [5]:
#making predictions for test directory

for k in range(0,400):
    t='{0:03}'.format(k)
    image=Image.open(test_dir+'/'+str(t)+'.jpg')
    im = np.array(image)/255.
    im = im.reshape(1, im.shape[0], im.shape[1], im.shape[2])
    prediction = model.predict_classes(im)
    if prediction == [0]:
        sub.label[k]='cat'
    else:
        sub.label[k]='dog'
        
sub.to_csv('out.csv', index=False)

/home/denis/anaconda3/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
/home/denis/anaconda3/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
